<a href="https://colab.research.google.com/github/sayarghoshroy/Hate-Speech-Detection/blob/master/dataset_creation/tweet_processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import pandas as pd
import xlrd
import re
import pickle
import csv

In [28]:
# Uncomment if you're running it for the first time
# !pip install ekphrasis
from ekphrasis.classes.segmenter import Segmenter
# to leverage word statistics from Twitter
seg_tw = Segmenter(corpus = "twitter")

Reading twitter - 1grams ...
Reading twitter - 2grams ...


In [29]:
# Uncomment if you're running it for the first time
# !pip install tweet-preprocessor
import preprocessor as tweet_proc

In [30]:
# Uncomment if you're running it for the first time
# !pip install emot
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

#### *Raw Datasets are hosted [here](https://drive.google.com/drive/folders/1TuHRQQ41lK9oXJhlhsRiMjMnczuKn_kF?usp=sharing).*

In [31]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount = True)

In [32]:
def make_list(proc_obj):
  if proc_obj == None:
    return []
  
  store = []
  for unit in proc_obj:
    store.append(unit.match)
  
  return store

def emotext(text):
    for emot in UNICODE_EMO:
        text = text.replace(emot, "_".join(UNICODE_EMO[emot].replace(",", "").replace(":", "").split()))
    return text

In [33]:
# For 2020 Datasets

is_hindi = 0

# For Train Data
# datatype = "train"
# For English
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_data/english.xlsx"

# For Hindi
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_data/hindi.xlsx"
# is_hindi = 1

# For German
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_data/german.xlsx"

# For Test Data
datatype = "train"
# For English
file_name = "data/golden.csv"

# For Hindi
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_test_data/hindi_test_1509.csv"
# is_hindi = 1

# For German
# file_name = "/content/drive/My Drive/HASOC_raw_data/2020_test_data/german_test_1509.csv"

In [34]:
# Initializing Lists
datapoints_count = 0
see_index = True

tweets = []
raw_tweet_texts = []
tokenized_tweets = []
hashtags = []
smileys = []
emojis = []
urls = []
mentions = []
numbers = []
reserveds = []
user_ID = []
task_1_labels = []
task_2_labels = []
tweet_ids = []
hasoc_ID = []

In [35]:
def strip_list(listie):
  stripped = []
  for item in listie:
    stripped.append(item.strip())
  return stripped

def hindi_clean(line, parse_obj):
  # beta
  tokens = line.replace(":", " : ").replace(",", " , ").replace(";", " ; ").split(" ")
  valid_stri = ""

  for raw_token in tokens:
    token = raw_token.strip()
    if token in strip_list(make_list(parse_obj.hashtags)):
      continue
    if token in strip_list(make_list(parse_obj.smileys)):
      continue
    if token in strip_list(make_list(parse_obj.emojis)):
      continue
    if token in strip_list(make_list(parse_obj.urls)):
      continue
    if token in strip_list(make_list(parse_obj.mentions)):
      continue
    if token in strip_list(make_list(parse_obj.numbers)):
      continue
    if token in strip_list(make_list(parse_obj.reserved)):
      continue
    valid_stri = valid_stri + " " + token
  return valid_stri.strip()

In [36]:
if datatype == 'train':
#     workbook = xlrd.open_workbook(file_name)
#     sheet = workbook.sheet_by_index(0)

#     for row in range(sheet.nrows):
#         line = sheet.row_values(row)

    file = open(file_name, 'r')
    file_reader = csv.reader(file, delimiter = ",")
    flag = 0
    for line in file_reader:
        if see_index == True:
            see_index = False
            continue
#         print(line)
#         flag += 1
#         if flag == 2:
#             break
        datapoints_count += 1
#         tweet_ids.append(line[1])
        task_1_labels.append(line[1])
#         task_2_labels.append(line[3])
#         user_ID.append(line[3])
        tweets.append(line[0].replace("\n", " "))

        parse_obj = tweet_proc.parse(line[0].replace("\n", " "))
        tokenized_tweets.append(tweet_proc.tokenize(line[0].replace("\n", " ")))
        hashtags.append(strip_list(make_list(parse_obj.hashtags)))
        smileys.append(strip_list(make_list(parse_obj.smileys)))
        emojis.append(strip_list(make_list(parse_obj.emojis)))
        urls.append(strip_list(make_list(parse_obj.urls)))
        mentions.append(strip_list(make_list(parse_obj.mentions)))
        numbers.append(strip_list(make_list(parse_obj.numbers)))
        reserveds.append(strip_list(make_list(parse_obj.reserved)))

        if is_hindi == 0:
          raw_tweet_texts.append(tweet_proc.clean(line[0].replace("\n", " ")))
        else:
          raw_tweet_texts.append(hindi_clean(line[0].replace("\n", " "), parse_obj))

    print("Number of Datapoints: " + str(datapoints_count))

Number of Datapoints: 532


In [37]:
if datatype == 'test':
    file = open(file_name, 'r')
    file_reader = csv.reader(file, delimiter = ",")
    for line in file_reader:
        if see_index == True:
            see_index = False
            continue

        datapoints_count += 1
        tweet_ids.append(line[0])
        task_1_labels.append(line[2])
        task_2_labels.append(line[3])
        hasoc_ID.append(line[4])
        tweets.append(line[1].replace("\n", " "))

        parse_obj = tweet_proc.parse(line[1].replace("\n", " "))
        tokenized_tweets.append(tweet_proc.tokenize(line[1].replace("\n", " ")))
        hashtags.append(strip_list(make_list(parse_obj.hashtags)))
        smileys.append(strip_list(make_list(parse_obj.smileys)))
        emojis.append(strip_list(make_list(parse_obj.emojis)))
        urls.append(strip_list(make_list(parse_obj.urls)))
        mentions.append(strip_list(make_list(parse_obj.mentions)))
        numbers.append(strip_list(make_list(parse_obj.numbers)))
        reserveds.append(strip_list(make_list(parse_obj.reserved)))

        if is_hindi == 0:
          raw_tweet_texts.append(tweet_proc.clean(line[1].replace("\n", " ")))
        else:
          raw_tweet_texts.append(hindi_clean(line[1].replace("\n", " "), parse_obj))

    print("Number of Datapoints: " + str(datapoints_count))

In [38]:
# Viewing Created Dataset
display_size = 10
start = 100

print("Tweets:")
print(tweets[start: start + display_size])

print("Raw Texts:")
print(raw_tweet_texts[start: start + display_size])

print("Hashtags:")
print(hashtags[start: start + display_size])

print("Smileys:")
print(smileys[start: start + display_size])

print("Emojis:")
print(emojis[start: start + display_size])

print("Urls:")
print(urls[start: start + display_size])

print("Mentions:")
print(mentions[start: start + display_size])

print("Numbers:")
print(numbers[start: start + display_size])

print("Reserved Words:")
print(reserveds[start: start + display_size])

print("Task Labels:")
print(task_1_labels[start: start + display_size])
print(task_2_labels[start: start + display_size])

Tweets:
['Nowadays IT, ED raids are based on scrutiny of Twitter accounts not buisness ledger accounts? #TaxChorBollywood  #FarmersCallBJPboycott https://t.co/9cttN3yrTF', "100 Days of India's Voice, In this 100 Days you saw Farmers Courage, Patience, Prudent, Peace. On other hand you saw Govt's Ego, Torture &amp; Injustice. Congratulations to PM Modi for Acche Din you delivered to India.  #FarmersCallBJPboycott #FarmersProtests #100DaysOfBJParrogance https://t.co/WELIghNTCk", 'No media showing this !!   Today marks 100 days of protests - farmers raising slogans at KMP expressway, Harayana.  Hope "Hum Do, Hamare Do" are listening to the voice of our farmers.   #100DaysOfBJParrogance  #FarmersCallBJPboycott https://t.co/iwMsM1IPJa', 'BJP is parasite of India... #FarmersCallBJPboycott', 'Is Modi the worst PM of India ever seen ?  #100DaysOfBJParrogance #FarmersCallBJPboycott', '100 days of protest and 248 deaths. The spirit, courage, and determination of our farmers are unbreakable!   #F

### Example
#### Raw Tweet Text
'RT @jeonggukpics: Don’t disturb please, he is enjoying his snacks while making those little dance 😭😂😂😭💜  #BBMAsTopSocial BTS #JUNGKOOK #정국…'
#### Clean Text
': Dont disturb please, he is enjoying his snacks while making those little dance BTS'

#### Emojis
'['😭', '😂', '😂', '😭', '💜']'

#### Hashtags
''#BBMAsTopSocial', '#JUNGKOOK', '#정국''

In [39]:
# Generating Emoji Texts
emoji_texts = []

for emo_list in emojis:
  texts = []
  for emoji in emo_list:
    text = emotext(emoji)
    texts.append(text.replace("_", " "))
  emoji_texts.append(texts)

print("Emoji Descriptions:")
print(emoji_texts[100: 200])

Emoji Descriptions:
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['writing hand'], [], [], ['face with stuck-out tongue & winking eye'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], ['dollar banknote', 'money with wings'], ['clapping hands', 'medium-light skin tone'], [], [], [], [], [], [], [], [], [], [], []]


In [40]:
# Segmenting Hashtags
segmented_hashtags = []

for hashset in hashtags:
  segmented_set = []
  for tag in hashset:
    word = tag[1: ]
    # removing the hash symbol
    segmented_set.append(seg_tw.segment(word))
  segmented_hashtags.append(segmented_set)

print("Segmented Hashtags:")
print(segmented_hashtags[0: 5])

Segmented Hashtags:
[['arnab goswami'], ['arnab goswami'], ['mumbai', 'arnab goswami', 'bombay high court'], ['national', 'arnab goswami', 'east mojo'], ['arnab goswami']]


In [41]:
name = 'data/extracted_golden.pickle'
dickie = {}
# dickie['tweet_id'] = tweet_ids
dickie['task_1'] = task_1_labels
# dickie['task_2'] = task_2_labels
# dickie['hasoc_id'] = hasoc_ID
dickie['full_tweet'] = tweets
dickie['tweet_raw_text'] = raw_tweet_texts
dickie['hashtags'] = hashtags
dickie['smiley'] = smileys
dickie['emoji'] = emojis
dickie['url'] = urls
dickie['mentions'] = mentions
dickie['numerals'] = numbers
dickie['reserved_word'] = reserveds
dickie['emotext'] = emoji_texts
dickie['segmented_hash'] = segmented_hashtags
# dickie['user_ID'] = user_ID
with open(name, 'wb') as f:
  pickle.dump(dickie, f)

In [42]:
with open(name, 'rb') as f:
  try_dict = pickle.load(f)

sizes = []
for key in try_dict.keys():
  sizes.append(len(try_dict[key]))

# Verifying if all sizes are equal
print(sizes)

[532, 532, 532, 532, 532, 532, 532, 532, 532, 532, 532, 532]


In [17]:
# ^_^ Thank You